In [1]:
import pandas as pd
import numpy as np
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from sklearn.mixture import DPGMM
from tabulate import tabulate

In [2]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file
from functions import *

In [3]:
df = pd.read_csv("../data/kaggle_data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/kaggle_data/upcoming-event.csv')
len(df)



4804

In [4]:
num_upcoming_fights = len(df_upcoming)

In [5]:
df = df_upcoming.append(df)
len(df)

4815

In [6]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
def return_finish_type(winner, finish):
    #print(winner, finish)
    #Why overcomplicate things?  We can just use a few if statements
    if winner == 'Red':
        #print("HI")
        if finish in ['U-DEC', 'S-DEC', 'M-DEC']:
            return ('Red - DEC')
        if finish in ['SUB']:
            return('Red - SUB')
        if finish in ['KO/TKO', 'DQ']:
            return('Red - KO/TKO')
    if winner == 'Blue':
        if finish in ['U-DEC', 'S-DEC', 'M-DEC']:
            return ('Blue - DEC')
        if finish in ['SUB']:
            return('Blue - SUB')
        if finish in ['KO/TKO', 'DQ']:
            return('Blue - KO/TKO')
        
    #Test for NaN
    if finish != finish:
        return('')
    
    if finish == 'Overturned':
        return('')
    
    
    return ('error')

In [8]:
#This calls for the power of lambda!
df['finish_type'] = df.apply(lambda x: return_finish_type(x['Winner'], x['finish']), axis=1)

In [9]:
finish_list = ['Red - DEC', 'Red - SUB', 'Red - KO/TKO', 'Blue - DEC', 'Blue - SUB', 'Blue - KO/TKO']

#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.

for f in range(len(finish_list)):
    mask = df['finish_type'] == finish_list[f]
    df['label'][mask] = f
    
#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

<ipython-input-9-af8f7542de83>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = f


In [10]:
df.rename(columns={'r_dec_odds': 'Red - DEC', 'r_sub_odds': 'Red - SUB', 'r_ko_odds': 'Red - KO/TKO',
                'b_dec_odds': 'Blue - DEC', 'b_sub_odds': 'Blue - SUB', 'b_ko_odds': 'Blue - KO/TKO'}, inplace=True)

In [11]:
df_label = df['label']
df_odds = df[finish_list]

In [12]:
df_train = df[num_upcoming_fights:]
odds_train = df_odds[num_upcoming_fights:]
label_train = df_label[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = df_odds[:num_upcoming_fights]
label_test = df_label[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

11
11
11
4804
4804
4804


In [13]:
weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_round_time,total_fight_time_secs,Red - DEC,Blue - DEC,Red - SUB,Blue - SUB,Red - KO/TKO,Blue - KO/TKO,finish_type,label
0,Cory Sandhagen,TJ Dillashaw,-210,175,47.619048,175.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,Blue,...,5:00,1500.0,500.0,500.0,600.0,1100.0,110.0,400.0,Blue - DEC,3.0
1,Kyler Phillips,Raulian Paiva,-320,250,31.250000,250.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,Blue,...,5:00,900.0,-125.0,500.0,1000.0,1000.0,400.0,900.0,Blue - DEC,3.0
2,Darren Elkins,Darrick Minner,130,-155,130.000000,64.516129,2021-07-24,"Las Vegas, Nevada, USA",USA,Red,...,3:48,528.0,275.0,250.0,600.0,240.0,800.0,600.0,Red - KO/TKO,2.0
3,Miranda Maverick,Maycee Barber,-155,135,64.516129,135.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,Blue,...,5:00,900.0,120.0,275.0,650.0,1400.0,750.0,450.0,Blue - DEC,3.0
4,Adrian Yanez,Randy Costa,-240,195,41.666667,195.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,Red,...,2:11,431.0,380.0,500.0,1100.0,1600.0,-110.0,450.0,Red - KO/TKO,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4799,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0:44,44.0,NaN,NaN,NaN,NaN,NaN,NaN,Blue - KO/TKO,5.0
4800,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2:01,121.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - KO/TKO,2.0
4801,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0:47,47.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - KO/TKO,2.0
4802,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,5:00,900.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - DEC,0.0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_round_time,total_fight_time_secs,Red - DEC,Blue - DEC,Red - SUB,Blue - SUB,Red - KO/TKO,Blue - KO/TKO,finish_type,label
0,Uriah Hall,Sean Strickland,170,-200,170.000000,50.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,575.0,200.0,2500.0,800.0,300.0,150.0,,NaN
1,Kyung Ho Kang,Rani Yahya,-120,100,83.333333,100.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,150.0,400.0,850.0,225.0,650.0,1300.0,,NaN
2,Cheyanne Buys,Gloria de Paula,-165,145,60.606061,145.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,-115.0,225.0,1000.0,1400.0,800.0,900.0,,NaN
3,Niklas Stolze,Jared Gooden,-180,155,55.555556,155.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,120.0,400.0,750.0,1100.0,350.0,500.0,,NaN
4,Bryan Barberena,Jason Witt,-275,220,36.363636,220.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,250.0,550.0,550.0,750.0,130.0,850.0,,NaN
5,Collin Anglin,Melsik Baghdasaryan,125,-145,125.000000,68.965517,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,250.0,240.0,1000.0,900.0,550.0,225.0,,NaN
6,Chris Gruetzemacher,Rafa Garcia,250,-320,250.000000,31.250000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,450.0,165.0,1700.0,200.0,900.0,400.0,,NaN
7,Danny Chavez,Kai Kamaka,100,-120,100.000000,83.333333,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,250.0,160.0,1600.0,1400.0,330.0,500.0,,NaN
8,Jinh Yu Frey,Ashley Yoder,130,-150,130.000000,66.666667,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,180.0,100.0,900.0,750.0,1400.0,1400.0,,NaN
9,Ryan Benoit,Zarrukh Adashev,-135,115,74.074074,115.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,165.0,250.0,1000.0,1800.0,350.0,400.0,,NaN


In [15]:
with open('../data/production_models_mov.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice automatically:
model_choice = 0
best_score = models[4][0]
for n in range(len(models[0])):
    print(n)
    if (float(models[4][n]) > float((best_score))):
        print("YES")
        best_score = models[4][n]
        model_choice = n

print(f"The model choice is {model_choice}")
print(models)

###REVERT THIS
#model_choice = 0

###


model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

0
1
The model choice is 0
[['model_2021_07_15', 'model_14'], ['LinearDiscriminantAnalysis()', 'LogisticRegressionCV(random_state=75)'], ['["R_Women\'s Featherweight_rank", \'B_Lightweight_rank\', \'R_Featherweight_rank\', \'R_Light Heavyweight_rank\', \'total_title_bout_dif\', \'R_losses\', \'B_win_by_Submission\', \'Blue - DEC\', \'B_draw\', \'R_Middleweight_rank\', \'B_avg_SIG_STR_pct\', "B_Women\'s Strawweight_rank", \'age_dif\', \'B_Middleweight_rank\', \'avg_sub_att_dif\', \'avg_td_dif\', \'B_avg_SUB_ATT\', "R_Women\'s Flyweight_rank", \'R_Pound-for-Pound_rank\', \'R_Reach_cms\', \'R_wins\', \'B_ev\', \'R_Weight_lbs\', \'sub_dif\', \'R_ev\', \'B_avg_TD_landed\']', '[\'R_win_by_TKO_Doctor_Stoppage\', \'R_win_by_Decision_Majority\', \'R_current_lose_streak\', \'R_draw\', \'R_avg_TD_pct\', \'B_Light Heavyweight_rank\', \'R_Welterweight_rank\', \'R_avg_SUB_ATT\', "B_Women\'s Flyweight_rank", \'R_Lightweight_rank\', \'R_total_rounds_fought\', \'R_Light Heavyweight_rank\', \'B_win_by_De

In [16]:
def display_bets(df, min_ev, finish_list, save_to_html=False):
    bet_list = []
    print("min_ev = " + str(min_ev))
    for index, row in df.iterrows():
        print()
        print()
        print(f"{row['t1_name']} (RED) vs. {row['t2_name']} (BLUE)")
        for f in finish_list:
            ev = get_bet_ev(row[f], row[f+'_prob'])
            if ev > min_ev:
                print(f"BET: {f}. ODDS: {row[f]}. PROB: {row[f+'_prob']}.  EV: {ev}")
                bet_list.append([row['t1_name'], row['t2_name'], f, row[f]])
                
    #print(bet_list)
    export_bets = pd.DataFrame(bet_list, columns=['Red', 'Blue', 'Bet', 'Odds'])
    if save_to_html:
        export_bets.to_csv('bets.csv', index=False)

In [17]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, finish_list, verbose=True, save_to_html=False):
    fighters_test = df[['R_fighter', 'B_fighter']]
    
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    #print(labels_train)
    print(len(df_train))
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]
    
    labels_train = labels_train.dropna()
    odds_train = odds_train[odds_train.index.isin(labels_train.index)]     
    df_train = df_train[df_train.index.isin(labels_train.index)] 

    
    
    #input_model.fit_transform(df_train, labels_train)
    #probs = input_model.predict_proba(df_test)
    
    
    
    scaler = StandardScaler()
    scaled_train = scaler.fit_transform(df_train)

    input_model.fit(scaled_train, labels_train)

    #print(scaled_train)
    
    scaled_test = scaler.transform(df_test)

    probs = input_model.predict_proba(scaled_test)    

    
    
    
    
    
    
    
    
    
    prob_label_list = []
    for f in finish_list:
        prob_label_list.append(f + '_prob')
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #print(probs)
    #print()
    #print()
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, 0], odds_test[:, 1],
                            odds_test[:, 2], odds_test[:, 3],odds_test[:, 4], odds_test[:, 5],
                            probs[:, 0], probs[:, 1],probs[:, 2], probs[:, 3],probs[:, 4], probs[:, 5]))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name'] + finish_list + prob_label_list)

    display_bets(ev_prepped_df, input_ev, finish_list, save_to_html=save_to_html)
    
    return(ev_prepped_df)

In [18]:
output_df = get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, finish_list, verbose = True, save_to_html=True)

3376
min_ev = 0


Uriah Hall (RED) vs. Sean Strickland (BLUE)
BET: Red - DEC. ODDS: 575.0. PROB: 0.27208590754688206.  EV: 83.6579875941454


Kyung Ho Kang (RED) vs. Rani Yahya (BLUE)
BET: Blue - SUB. ODDS: 225.0. PROB: 0.7309028883054967.  EV: 137.5434386992864


Cheyanne Buys (RED) vs. Gloria de Paula (BLUE)


Niklas Stolze (RED) vs. Jared Gooden (BLUE)


Bryan Barberena (RED) vs. Jason Witt (BLUE)
BET: Blue - DEC. ODDS: 550.0. PROB: 0.16822651306063394.  EV: 9.347233489412062
BET: Blue - SUB. ODDS: 750.0. PROB: 0.1342016635194112.  EV: 14.071413991499526


Collin Anglin (RED) vs. Melsik Baghdasaryan (BLUE)
BET: Red - DEC. ODDS: 250.0. PROB: 0.4281630052274479.  EV: 49.85705182960675


Chris Gruetzemacher (RED) vs. Rafa Garcia (BLUE)
BET: Red - DEC. ODDS: 450.0. PROB: 0.18668507918584393.  EV: 2.67679355221415
BET: Blue - KO/TKO. ODDS: 400.0. PROB: 0.2707500221707475.  EV: 35.37501108537376


Danny Chavez (RED) vs. Kai Kamaka (BLUE)
BET: Red - DEC. ODDS: 250.0. PROB: 0.41407344226570

In [19]:
output_df

,t1_name,t2_name,Red - DEC,Red - SUB,Red - KO/TKO,Blue - DEC,Blue - SUB,Blue - KO/TKO,Red - DEC_prob,Red - SUB_prob,Red - KO/TKO_prob,Blue - DEC_prob,Blue - SUB_prob,Blue - KO/TKO_prob
0,Uriah Hall,Sean Strickland,575.0,2500.0,300.0,200.0,800.0,150.0,0.272086,0.020255,0.147460,0.177013,0.083445,0.299741
1,Kyung Ho Kang,Rani Yahya,150.0,850.0,650.0,400.0,225.0,1300.0,0.118506,0.008851,0.029251,0.099381,0.730903,0.013109
2,Cheyanne Buys,Gloria de Paula,-115.0,1000.0,800.0,225.0,1400.0,900.0,0.399859,0.080250,0.106044,0.270279,0.055230,0.088338
3,Niklas Stolze,Jared Gooden,120.0,750.0,350.0,400.0,1100.0,500.0,0.385008,0.080647,0.172226,0.195741,0.035906,0.130472
4,Bryan Barberena,Jason Witt,250.0,550.0,130.0,550.0,750.0,850.0,0.242998,0.130578,0.254445,0.168227,0.134202,0.069550
5,Collin Anglin,Melsik Baghdasaryan,250.0,1000.0,550.0,240.0,900.0,225.0,0.428163,0.065599,0.076328,0.235050,0.029312,0.165548
6,Chris Gruetzemacher,Rafa Garcia,450.0,1700.0,900.0,165.0,200.0,400.0,0.186685,0.024072,0.099251,0.329102,0.090140,0.270750
7,Danny Chavez,Kai Kamaka,250.0,1600.0,330.0,160.0,1400.0,500.0,0.414073,0.056717,0.086996,0.313726,0.042870,0.085616
8,Jinh Yu Frey,Ashley Yoder,180.0,900.0,1400.0,100.0,750.0,1400.0,0.322000,0.075248,0.072756,0.346509,0.080987,0.102500
9,Ryan Benoit,Zarrukh Adashev,165.0,1000.0,350.0,250.0,1800.0,400.0,0.404392,0.062901,0.102254,0.264158,0.056542,0.109753
